# Clifpy must be installed for this

In [1]:
%%bash

if ! python -m pip show clifpy > /dev/null 2>&1; then
    python -m pip install clifpy
fi

In [2]:
from pathlib import Path
import yaml
import pandas as pd

# -----------------------------
# Find project root (directory containing "config/")
# -----------------------------
def find_project_root(start_path=None, target_dir="config"):
    start_path = Path(start_path or Path.cwd()).resolve()
    for parent in [start_path, *start_path.parents]:
        if (parent / target_dir).is_dir():
            return parent
    raise FileNotFoundError(f'Could not find "{target_dir}" directory in parents')

project_root = find_project_root()

# -----------------------------
# Read YAML config
# -----------------------------
with open(project_root / "config" / "config.yaml", "r") as f:
    config = yaml.safe_load(f)

tables_location  = config["tables_location"]
project_location = config["project_location"]
site             = config["institution"]

# -----------------------------
# Load hospital block key and extract hospitalization IDs
# -----------------------------
hospital_block_key = pd.read_csv(
    Path(project_location) / "private_tables" / "hospital_block_key.csv"
)

hospitalizations = (
    hospital_block_key["hospitalization_id"]
    .astype(str)
    .drop_duplicates()
    .tolist()
)

In [3]:
from clifpy.utils import compute_ase

# Compute ASE for specific hospitalizations
ase_results = compute_ase(
    hospitalization_ids= hospitalizations,
    data_directory= tables_location,
    filetype='parquet',
    timezone='UTC'
)


ASE COMPUTATION STARTED
Processing 8959 hospitalizations
Configuration: RIT=True, Lactate=False

=== Processing Blood Cultures ===
  Loading cultures...
📢 Initialized microbiology_culture table
📢 Data directory: /projects/CLIF-analysis/critill_rclif_2_1/critill_rclif_2_1_tables
📢 File type: parquet
📢 Timezone: UTC
📢 Output directory: /projects/CLIF-analysis/imc_arf_hpc/code/output
📢 Loaded schema from /home/sgoldfa3/.local/lib/python3.9/site-packages/clifpy/schemas/microbiology_culture_schema.yaml
📢 Loaded outlier configuration
    Loaded 23,126 rows, memory cleaned
Total cultures loaded: 9866
Fluid categories: {'blood_buffy': 9866}

Blood cultures per patient statistics:
  Patients with 1 blood culture: 2588
  Patients with 2+ blood cultures: 2360
  Max blood cultures per patient: 33

=== Calculating QAD ===
  Loading antibiotics_raw...
📢 Initialized medication_admin_intermittent table
📢 Data directory: /projects/CLIF-analysis/critill_rclif_2_1/critill_rclif_2_1_tables
📢 File type: p

/home/sgoldfa3/.local/lib/python3.9/site-packages/clifpy/utils/ase.py:1559: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  keep_mask = all_sepsis.groupby("hospitalization_id", group_keys=False).apply(filter_group)


  ASE episodes after RIT: 3419


/home/sgoldfa3/.local/lib/python3.9/site-packages/clifpy/utils/ase.py:1804: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ase_df = pd.concat([ase_df, no_bc_df], ignore_index=True)
/home/sgoldfa3/.local/lib/python3.9/site-packages/clifpy/utils/ase.py:1804: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ase_df = pd.concat([ase_df, no_bc_df], ignore_index=True)
/home/sgoldfa3/.local/lib/python3.9/site-packages/clifpy/utils/ase.py:1804: FutureWarning: The behavior of DataFrame concatenation with empt


ASE COMPUTATION COMPLETE
Total ASE events: 3419
Total hospitalizations processed: 8959



In [4]:
ase_results.to_csv(
    Path(project_location) / "private_tables" / "sepsis_indicators.csv",
    index=False
)